In [17]:
import pandas as pd
df = pd.read_csv('twitter_training.csv')
df.columns = ['id','topic','sentiment','text']
df.head()

,id,topic,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         74681 non-null  int64 
 1   topic      74681 non-null  object
 2   sentiment  74681 non-null  object
 3   text       73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [19]:
df['sentiment'].value_counts()

sentiment
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [20]:
df.isnull().sum()

id             0
topic          0
sentiment      0
text         686
dtype: int64

In [21]:
df.duplicated().sum()

2700

In [22]:
df = df.dropna(subset=['text'])


In [23]:
df = df.drop_duplicates()

In [25]:
df['sentiment'] = df['sentiment'].replace('Irrelevant','Neutral')


In [48]:
import re
def clean_text(text):
    if isinstance(text,str):  # Make sure input is a string
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
        text = re.sub(r'@\w+', '', text)  # Remove mentions (@username)
        text = re.sub(r'#\w+', '', text)  # Remove hashtags (#tag)
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d+', '', text)  # Remove digits
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        return text
    else: 
        return ""


In [49]:
df['clean_text'] = df['text'].apply(clean_text)

In [50]:
from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['sentiment']
X_text_train,X_text_test,y_train,y_test = train_test_split(
    X, y, test_size=0.2,random_state=42
)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train = vectorizer.fit_transform(X_text_train)
X_test = vectorizer.transform(X_text_test)



In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [53]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train , y_train)


LogisticRegression(max_iter=1000)

In [54]:
y_pred = model.predict(X_test)
print(y_pred)
print(y_test)

[1 2 1 ... 1 1 1]
[1 1 2 ... 1 2 1]


In [59]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy", accuracy_score(y_test,y_pred))
print("\nClassification Report:\n",classification_report(y_test,y_pred,target_names=target_names))
print("\nConfusin Matrix:\n",confusion_matrix(y_test,y_pred))

Accuracy 0.7251413020724304

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.73      0.75      4433
           1       0.70      0.77      0.74      5987
           2       0.72      0.65      0.68      3911

    accuracy                           0.73     14331
   macro avg       0.73      0.72      0.72     14331
weighted avg       0.73      0.73      0.72     14331


Confusin Matrix:
 [[3236  906  291]
 [ 648 4626  713]
 [ 327 1054 2530]]


In [64]:
results = pd.DataFrame({
    'Tweet': X_text_test.values,
    'True Sentiment': le.inverse_transform(y_test),
    'Predicted sentiment': le.inverse_transform(y_pred)
})
results.sample(10)

,Tweet,True Sentiment,Predicted sentiment
3095,hey response,Negative,Neutral
129,list is trash without minkah,Negative,Negative
8059,monday johnson johnson stops selling talc by b...,Neutral,Neutral
2181,cause why actual actual checkers received stat...,Negative,Neutral
8326,can you guys look into why im constantly getti...,Negative,Negative
5755,ps video is leaked ps devkit xhardhempusnet to...,Neutral,Neutral
6491,beautiful,Positive,Positive
1827,to thank you to the very brave and gamers of h...,Neutral,Neutral
13002,at the same time despite the fact that there a...,Neutral,Neutral
8608,literally no,Negative,Negative
